In [1]:
import mbuild as mb
import foyer
import parmed
import subprocess

# Step 1: Accessing the TraPPE website

Head to the [TraPPE website](http://chem-siepmann.oit.umn.edu/siepmann/trappe/index.html) and search for Ethane (C3H8). 
This page contains extensive information on how to implement the TraPPE force field. 

More importantly, the bottom of the page contains some structure PDB and foyer forcefield XML files. 
We will download the "Monomer 1" PDB and Foyer XML files.

# Step 2: Building the mBuild compound
To implement non-atomistic compounds and the bond-less PDB files from the TraPPE website, some modifications will be necessary after creating our `mb.Compound`

For extensibility purposes, these functions could be easily wrapped into an mBuild class.

In [2]:
cmpd = mb.load('TraPPE_UA_3_propane_monomer1.pdb')
cmpd.name = 'Pro'

The PDB has some awkward naming conventions that manifest in the mBuild particles.
To remedy this, we will convert the particles' names to follow the non-atomistic convention, 
where particle names are prefaced with an underscore.

Furthermore, the PDB file does not contain bond information, so we will manually specify bonds in mBuild.

In [3]:
particles = [a for a in cmpd.particles()]
particles

[<1CH3 pos=( 2.2350, 2.7578, 1.2698), 0 bonds, id: 4512721440>,
 <CH2 pos=( 2.2991, 2.6213, 1.2385), 0 bonds, id: 4512805272>,
 <2CH3 pos=( 2.1983, 2.5050, 1.2329), 0 bonds, id: 4786419640>]

In [4]:
particles[0].name = "_" + particles[0].name[1:]
particles[1].name = "_" + particles[1].name
particles[2].name = "_" + particles[2].name[1:]
cmpd.add_bond((particles[0], particles[1]))
cmpd.add_bond((particles[1], particles[2]))
particles

[<_CH3 pos=( 2.2350, 2.7578, 1.2698), 0 bonds, id: 4512721440>,
 <_CH2 pos=( 2.2991, 2.6213, 1.2385), 0 bonds, id: 4512805272>,
 <_CH3 pos=( 2.1983, 2.5050, 1.2329), 0 bonds, id: 4786419640>]

After "cleaning up" the mBuild compound, we can fill a box to generate a molecular system

In [5]:
box = mb.fill_box(cmpd, n_compounds=5, box=[5,5,5])

# Step 3: Applying the foyer force field
The foyer XML provided on the TraPPE website requires some modification to fulfill the foyer XML schema.

Namely, the `ForceField` XML element should not have the attribute `model`. While this is useful information to document, this is not consistent with the current foyer XML schema, so the attribute needs to be removed.

Furthermore, the `HarmonicBondForce`, `HarmonicAngleForce`, and `RBTorsionForce` elements may include DOI information. Again, while this is useful information to document, this is not consistent with the current foyer XML schema, so the DOIs in these XML elements need to be removed. The DOIs in the `AtomTypes` section, however, can be kept.

The XML provided in this repo has already modified the TraPPE XML.

Note: The "canonical" TraPPE implementation utilizes *constrained bonds or fixed bond lengths*, but the listed bond constants within the XML are based on GAFF parametrization for flexible bonds. Please observe this feature within TraPPE to ensure simulation consistency. Additionally, observe that the TraPPE force field excludes 1-4 LJ and Coulombic interactions

In [6]:
ff = foyer.Forcefield(forcefield_files='TraPPE_UA_3_fully_flexible_propane.xml')

/Users/ayang41/Programs/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  'Creating custom element for {}'.format(element))
/Users/ayang41/Programs/foyer/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH2
  'Creating custom element for {}'.format(element))


In [7]:
structure = box.to_parmed(residues='Pro')
parametrized = ff.apply(structure)

/Users/ayang41/Programs/mbuild/mbuild/compound.py:2410: UserWarning: Guessing that "<_CH3 pos=( 0.6139, 1.2359, 2.7388), 0 bonds, id: 4786848320>" is element: "EP"
  atom, element))


After parametrizing our compound, we can save the structure to gromacs-suitable files.

In [8]:
parametrized.save('out.gro', overwrite=True)
parametrized.save('out.top', overwrite=True)

# Step 4: Running the simulation

As with most simulation engines, an input or control file is necessary. 
For gromacs, this is the MDP file. 

* As mentioned earlier in Step 3, TraPPE typically utilizes fixed bond lengths, so we make sure to specify `constraints = all-bonds`
* As specified on the TraPPE website, cutoffs are 1.4 nm, tail corrections for LJ interactions, and Ewald summations for Coulombic interactions


In [9]:
def write_gmx_mdp(filename):
    with open(filename, 'w') as f:
        f.write("""
title                     = NVT Equilibration
; Run parameters
integrator                = md        ; leap-frog integrator
nsteps                    = 5000     ; 2 * 5000 = 10ps
dt                        = 0.002     ; 2 fs

; Output control
nstxout                   = 500       ; Every 1.0 ps
nstvout                   = 500
nstenergy                 = 500
nstlog                    = 500

;Bond parameters
continuation              = no
constraint_algorithm    = lincs
constraints             =   all-bonds
lincs_iter              = 1
lincs_order             = 4

; Neighbor searching
cutoff-scheme           = Verlet
nstype                  = grid
nstlist                 = 10
rcoulomb                = 1.4
rvdw                    = 1.4

; Electrostatics
coulombtype             = PME
pme_order               = 4
fourierspacing          = 0.16

; Temperature coupling
tcoupl                  = nose-hoover
tc-grps                 = system
tau_t                   = 0.4  
ref_t                   = 300  

; Pressure coupling
pcoupl                 = no

;Periodic boundary conditions
pbc                     = xyz

;Dispersion correction
DispCorr                = EnerPres

;Velocity generation
gen_vel                 = yes
gen_temp                = 300
gen_seed                = -1""")

In [10]:
write_gmx_mdp('nvt.mdp')
grompp_cmd = 'gmx grompp -f nvt.mdp -c out.gro -p out.top -maxwarn 1 -o nvt'
p = subprocess.Popen(grompp_cmd, shell=True, universal_newlines=True,
                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()
with open('grompp.out', 'w') as f:
    f.write(out)
with open('grompp.err', 'w') as f:
    f.write(err)
    
mdrun_cmd = 'gmx mdrun -deffnm nvt'
p = subprocess.Popen(mdrun_cmd, shell=True, universal_newlines=True,
                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = p.communicate()
with open('mdrun.out', 'w') as f:
    f.write(out)
with open('mdrun.err', 'w') as f:
    f.write(err)

# Step 5: Analysis

# Concluding Remarks
Some low-level routines for constructing a TraPPE molecular simulation have been provided. There is clear room for extensibility to build multiple, different mBuild compounds; to simulate different thermodynamic statepoints; and to do so in a very modular, reproducible fashion. Future work can include disseminating the entire TraPPE force field within an XML, validating dihedral fits within the TraPPE implementation, and adjusting the PDB/XML files on the TraPPE website to ensure MoSDeF consistency.